# 연습 제출 파일 제작

In [1]:
import argparse, glob, os, torch, warnings, time
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier
from xgboost import XGBClassifier
from tqdm import tqdm
from sklearn.metrics import f1_score ,roc_auc_score, recall_score
import time
#from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

import lightgbm as lgb

#parser = argparse.ArgumentParser(description = "ensemble_trainer")

## Training and evaluation path/lists, save path
#parser.add_argument('--csv_path0', type=str, default='/Data2/folder_ti/data/1st_1011_train.csv', help='1st classification file')
#parser.add_argument('--csv_path1', type=str, default='/Data2/folder_ti/data/1st_1011_test.csv', help='1st classification test file')
#parser.add_argument('--csv_path2', type=str, default='/Data2/folder_ti/data/tr_2nd_class_after_bank_cluster5.csv', help='2nd classifition file')
#parser.add_argument('--csv_path3', type=str, default='/Data2/folder_ti/data/te_2nd_class_after_bank_cluster5.csv', help='2nd classifition test file')
#parser.add_argument('--csv_path4', type=str, default='/Data2/folder_ti/data/submission/submission.csv', help='submission file')
#parser.add_argument('--save_path', type=str, default="exps/ensemble/ensemble(tab_xgb)_1st_2nd_grid_score.txt", help='Path to recording score')
## Model and Loss settings

print ('Available devices ', torch.cuda.device_count())
GPU_NUM = 4 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())

## Initialization
#warnings.simplefilter("ignore")
#torch.multiprocessing.set_sharing_strategy('file_system')
#args = parser.parse_args()

print('*'*50)
print('1st classication preprocessing,,,,,,')
print('*'*50)

train = pd.read_csv('/Data2/folder_ti/data/1st_1011_train.csv' ,index_col=0)
test = pd.read_csv('/Data2/folder_ti/data/1st_1011_test.csv' ,index_col=0)

Available devices  5
Device: cuda:4
Current cuda device: 4
**************************************************
1st classication preprocessing,,,,,,
**************************************************


In [2]:
def trans(x):
    if x == '노대출':
        return 0
    elif x == '대출':
        return 1
train['is_applied'] = train['is_applied'].apply(trans)

cat_col = ['purpose','income_type','rehabilitation','gender','employment_type','houseown_type']

cat_trans = dict()
for cat in cat_col :
    cat_trans[cat] = dict()
    type_ = list(set(train[cat].tolist()))
    for i, ty in enumerate(type_) :
        cat_trans[cat][ty] = i
        
cat_trans1 = dict()
for cat in cat_col :
    cat_trans1[cat] = dict()
    type_ = list(set(test[cat].tolist()))
    for i, ty in enumerate(type_) :
        cat_trans1[cat][ty] = i
        
        
for cat in cat_col :
    for k in tqdm(cat_trans[cat].keys()) :
        train[cat][train[cat]==k] = cat_trans[cat][k] 
        
for cat in cat_col :
    for k in tqdm(cat_trans1[cat].keys()) :
        test[cat][test[cat]==k] = cat_trans1[cat][k] 

types = train.dtypes
target = 'is_applied'
unused_col = ['set','user_id','application_id']
cat_col = ['purpose','income_type','rehabilitation','gender','employment_type','houseown_type']
categorical_dims =  {}
for col in train.columns:
    if col in cat_col :
        print(col, train[col].nunique())
        l_enc = LabelEncoder()
        train[col] = l_enc.fit_transform(train[col].values)
        categorical_dims[col] = len(l_enc.classes_)
    elif col not in unused_col+[target]:
        train.fillna(train.loc[:, col].mean(), inplace=True)


# Categorical Embedding을 위해 Categorical 변수의 차원과 idxs를 담음.
features = [ col for col in train.columns if col not in unused_col+[target]] 
cat_idxs = [ i for i, f in enumerate(features) if f in cat_col]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in cat_col]

train = train.astype({'income_type':'float32','purpose':'float32','employment_type':'float32','houseown_type':'float32'})
test = test.astype({'income_type':'float32','purpose':'float32','employment_type':'float32','houseown_type':'float32'})

print('****************** preprocessing end! ******************')


params_tab = {'n_d' : np.random.choice([4]),
         'n_steps' :np.random.choice([5]),        # tree에 depth
         'gamma': np.random.choice([1.5]),
         'momentum':np.random.choice([0.5]),
         'mask_type':np.random.choice(['sparsemax'])
         }

params_xgb = {'max_depth' : np.random.choice([6]),
              'subsample' : np.random.choice([.8]),
             'colsample_bytree' :np.random.choice([.8]),        # tree에 depth
             'n_estimators': np.random.choice([300])
             }

clf_xgb = XGBClassifier(**params_xgb,
                        learning_rate=0.2, 
                       objective='binary:logistic',
                       eval_metric = 'error@0.33',
                       tree_method='gpu_hist',
                       gpu_id=GPU_NUM)



clf_tab = TabNetClassifier(**params_tab, 
                       n_independent = 2, 
                       n_shared = 2, 
                       lambda_sparse = 1e-4, 
                       cat_idxs = cat_idxs, 
                       cat_dims = cat_dims,
                       cat_emb_dim = 1,
                       clip_value = 2,
                      optimizer_fn = torch.optim.Adam,
                       optimizer_params = {'lr':3e-2},
                       scheduler_params = {"gamma": 0.95,
                                         "step_size": 20},
                       scheduler_fn = torch.optim.lr_scheduler.StepLR,
                       epsilon = 1e-15,
                       device_name = device
                      )



X_train = train.loc[:,features].values
y_train = train.loc[:,target].values

X_test = test.loc[:,features].values
y_test = test.loc[:,target].values
                                                                                            #######

print('*'*20,' XGBoost 1st classification is stared! ','*'*20)
clf_xgb.fit(X_train, y_train)
print('*'*20,' XGBoost 1st classification is done! ','*'*20)


print('*'*20,' TabNet 1st classification is stared! ','*'*20)
clf_tab.fit(X_train=X_train, y_train=y_train,
    max_epochs=1, patience=20,
    batch_size=10240, virtual_batch_size=1280,
    num_workers=0,
    weights=1,
    drop_last=False
)

print('*'*20,' TabNet 1st classification is done! ','*'*20)


print('*'*20,' 1st classification is done! ','*'*20)


  0%|                                                                                                                       | 0/8 [00:00<?, ?it/s]/tmp/ipykernel_13151/1444815968.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[cat][train[cat]==k] = cat_trans[cat][k]
  0%|                                                                                                                       | 0/4 [00:00<?, ?it/s]/tmp/ipykernel_13151/1444815968.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[cat][train[cat]==k] = cat_trans[cat][k]
/tmp/ipykernel_13151/1444815968.py:27: SettingWithCopyWarning: 
A value 

purpose 8
gender 3
income_type 6
employment_type 4
houseown_type 4
rehabilitation 4
****************** preprocessing end! ******************
********************  XGBoost 1st classification is stared!  ********************


/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda:4
  warnings.warn(f"Device used : {self.device}")


********************  XGBoost 1st classification is done!  ********************
********************  TabNet 1st classification is stared!  ********************


/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.66893 |  0:00:11s
********************  TabNet 1st classification is done!  ********************
********************  1st classification is done!  ********************


In [3]:
def thres_(threshold, array) :
    pred = np.ceil(array[:,1]-threshold)
    return pred
def thres_2(threshold, array) :
    pred = np.ceil(array-threshold)
    return pred
                                                       
                                
print('*'*50)
print('1st Classication Soft Voting Ensemble is Proceeding ,,,,,,,,')
print('*'*50)


pred_prob_valid_xgb = clf_xgb.predict_proba(X_test)

pred_prob_valid_tab = clf_tab.predict_proba(X_test)


def proba_weighted_mean_best_pred(thres,x,y) :
    wm = (thres*x[:,1] + (100-thres)*y[:,1])/100
                                
    f1_best_thres_valid_ = 0.34

    return thres_2(f1_best_thres_valid_,wm)                                
                                
                                

pre_valid_soft = proba_weighted_mean_best_pred(0.1*100, pred_prob_valid_tab, pred_prob_valid_xgb)

                                

print('*'*50)
print('1st Classication Soft Voting Ensemble is Done!')
print('*'*50)

train_1st = train.loc[:,['application_id']+features]
valid_1st = test.loc[:,['application_id']+features]

train_1st_id_set = list(set(train_1st.application_id.to_list()))
valid_1st_id_set = list(set(valid_1st.application_id.to_list()))

valid_1st['pred'] = pre_valid_soft

valid_1st_pred = valid_1st.loc[:,['application_id','pred']]

test_1st_id_set = list(set(valid_1st[valid_1st['pred']==1].application_id.to_list()))
test_1st_else_id_set = list(set(valid_1st[valid_1st['pred']==0].application_id.to_list()))

**************************************************
1st Classication Soft Voting Ensemble is Proceeding ,,,,,,,,
**************************************************
**************************************************
1st Classication Soft Voting Ensemble is Done!
**************************************************


In [4]:
print('*'*50)
print('2nd Classication Data Prepare ,,,,,,,')
print('*'*50)


second = pd.read_csv('/Data2/folder_ti/data/tr_2nd_class_after_bank_cluster5.csv')
second_test = pd.read_csv('/Data2/folder_ti/data/te_2nd_class_after_bank_cluster5.csv')
    
second.drop(['purpose','employment_type','houseown_type','user_id','gender','company_year','loanapply_insert_time','insert_time'],axis=1,inplace=True)  #
second = pd.merge(second,train.loc[:,['application_id','purpose','employment_type','houseown_type']],on='application_id',how='left')
    
    
second_test.drop(['purpose','employment_type','houseown_type','user_id','gender','company_year','loanapply_insert_time','insert_time'],axis=1,inplace=True)  #
second_test = pd.merge(second_test,test.loc[:,['application_id','purpose','employment_type','houseown_type']],on='application_id',how='left')

first_only_col = ['application_id','past.is_applied','m_past.is_applied','seq','min.loan_rate','product_n','n.limit_over_desire','loan_limit_min.rate',
                 'diff2','limit_over_desire_product','yd']

second = pd.merge(second,train.loc[:,first_only_col],on='application_id', how='left')
second_test = pd.merge(second_test,test.loc[:,first_only_col],on='application_id', how='left')


train_2nd = second[second['application_id'].isin(train_1st_id_set)]
valid_2nd = second_test[second_test['application_id'].isin(valid_1st_id_set)]         ### 안될수도있음
    
test_2nd = second_test[second_test['application_id'].isin(test_1st_id_set)]
test_2nd_else = second_test[second_test['application_id'].isin(test_1st_else_id_set)]

assert valid_2nd.shape[0] == (test_2nd.shape[0]+test_2nd_else.shape[0]), '2nd Classification Valid vs Test,Test else 길이가 맞지 않습니다.'
    
    


cat_col = ['bank_id','purpose','income_type','employment_type','houseown_type','rehabilitation','rate_rank','bank_cluster','cluster_min_rate']


cat_trans = dict()
for cat in cat_col :
    cat_trans[cat] = dict()
    type_ = list(set(test_2nd[cat].tolist()))
    for i, ty in enumerate(type_) :
        cat_trans[cat][ty] = i
        

for cat in cat_col :
    for k in tqdm(cat_trans[cat].keys()) :
        train_2nd[cat][train_2nd[cat]==k] = cat_trans[cat][k]   
for cat in cat_col :
    for k in tqdm(cat_trans[cat].keys()) :
        valid_2nd[cat][valid_2nd[cat]==k] = cat_trans[cat][k]   
for cat in cat_col :
    for k in tqdm(cat_trans[cat].keys()) :
        test_2nd[cat][test_2nd[cat]==k] = cat_trans[cat][k]   
for cat in cat_col :
    for k in tqdm(cat_trans[cat].keys()) :
        test_2nd_else[cat][test_2nd_else[cat]==k] = cat_trans[cat][k] 



types = valid_2nd.dtypes
target = 'is_applied'
#unused_col = ['set','user_id','application_id','loanapply_insert_time','insert_time','gender','yearly_income',
#             'company_year','existing_loan_cnt','product_id','pred']
unused_col = ['set','application_id','pred','pred2','product_id']  # ,'loanapply_insert_time','insert_time','product_id'
cat_col = ['bank_id','purpose','income_type','employment_type','houseown_type','rehabilitation','rate_rank','bank_cluster','cluster_min_rate']
categorical_dims =  {}
for col in valid_2nd.columns:
    if col in cat_col :
        print(col, valid_2nd[col].nunique())
        l_enc = LabelEncoder()
        valid_2nd[col] = l_enc.fit_transform(valid_2nd[col].values)
        categorical_dims[col] = len(l_enc.classes_)
    elif col not in unused_col+[target]:
        valid_2nd.fillna(valid_2nd.loc[:, col].mean(), inplace=True)


# Categorical Embedding을 위해 Categorical 변수의 차원과 idxs를 담음.
features = [ col for col in valid_2nd.columns if col not in unused_col+[target]] 
cat_idxs = [ i for i, f in enumerate(features) if f in cat_col]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in cat_col]
#train_shuf = train_shuf.astype({'income_type':'int64','employment_type':'int64','houseown_type':'int64','purpose':'int64'})

train_2nd = train_2nd.astype({'income_type':'float32','purpose':'float32','employment_type':'float32','houseown_type':'float32'})
valid_2nd = valid_2nd.astype({'income_type':'float32','purpose':'float32','employment_type':'float32','houseown_type':'float32'})
test_2nd = test_2nd.astype({'income_type':'float32','purpose':'float32','employment_type':'float32','houseown_type':'float32'})
test_2nd_else = test_2nd_else.astype({'income_type':'float32','purpose':'float32','employment_type':'float32','houseown_type':'float32'})



X_train = train_2nd.loc[:,features].values
y_train = train_2nd.loc[:,target].values

X_valid = test_2nd.loc[:,features].values

X_valid_else = test_2nd_else.loc[:,features].values


print('*'*20,'2nd Classication Data Prepared!','*'*20)


print('*'*50)
print('2nd Classication Start!')
print('*'*50)

params_tab = {'n_d' : np.random.choice([4]),
         'n_steps' :np.random.choice([5]),        # tree에 depth
         'gamma': np.random.choice([1.5]),
         'momentum':np.random.choice([0.5]),
         'mask_type':np.random.choice(['sparsemax'])
         }

params_xgb = {'max_depth' : np.random.choice([6]),
              'subsample' : np.random.choice([.8]),
             'colsample_bytree' :np.random.choice([.8]),        # tree에 depth
             'n_estimators': np.random.choice([300])
             }

clf_xgb = XGBClassifier(**params_xgb,
                        learning_rate=0.2, 
                       objective='binary:logistic',
                       eval_metric = 'error@0.33',
                       tree_method='gpu_hist',
                       gpu_id=GPU_NUM)



clf_tab = TabNetClassifier(**params_tab, 
                       n_independent = 2, 
                       n_shared = 2, 
                       lambda_sparse = 1e-4, 
                       cat_idxs = cat_idxs, 
                       cat_dims = cat_dims,
                       cat_emb_dim = 1,
                       clip_value = 2,
                      optimizer_fn = torch.optim.Adam,
                       optimizer_params = {'lr':3e-2},
                       scheduler_params = {"gamma": 0.95,
                                         "step_size": 20},
                       scheduler_fn = torch.optim.lr_scheduler.StepLR,
                       epsilon = 1e-15,
                       device_name = device
                      )




print('*'*20,' XGBoost 2nd Classification is Stared ,,,,,,,,,, ','*'*20)
clf_xgb.fit(X_train, y_train)
print('*'*20,' XGBoost 2nd Classification is Done! ','*'*20)


print('*'*20,' TabNet 2nd Classification is Stared ,,,,,,,,,, ','*'*20)
clf_tab.fit(X_train=X_train, y_train=y_train,
    max_epochs=1, patience=20,
    batch_size=10240, virtual_batch_size=1280,
    num_workers=0,
    weights=1,
    drop_last=False
)
print('*'*20,' TabNet 2nd Classification is Done! ','*'*20)



**************************************************
2nd Classication Data Prepare ,,,,,,,
**************************************************


  0%|                                                                                                                      | 0/61 [00:00<?, ?it/s]/tmp/ipykernel_13151/2033497012.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_2nd[cat][train_2nd[cat]==k] = cat_trans[cat][k]
/tmp/ipykernel_13151/2033497012.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_2nd[cat][train_2nd[cat]==k] = cat_trans[cat][k]
/tmp/ipykernel_13151/2033497012.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-

 49%|█████████████████████████████████████████████████████▌                                                       | 30/61 [00:00<00:00, 93.94it/s]/tmp/ipykernel_13151/2033497012.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_2nd[cat][train_2nd[cat]==k] = cat_trans[cat][k]
/tmp/ipykernel_13151/2033497012.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_2nd[cat][train_2nd[cat]==k] = cat_trans[cat][k]
/tmp/ipykernel_13151/2033497012.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-

/tmp/ipykernel_13151/2033497012.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_2nd[cat][train_2nd[cat]==k] = cat_trans[cat][k]
/tmp/ipykernel_13151/2033497012.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_2nd[cat][train_2nd[cat]==k] = cat_trans[cat][k]
/tmp/ipykernel_13151/2033497012.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_2nd[cat][train_2nd[cat]==k] = cat_trans[cat][k]
 98%|█████████████████

  0%|                                                                                                                      | 0/80 [00:00<?, ?it/s]/tmp/ipykernel_13151/2033497012.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_2nd[cat][train_2nd[cat]==k] = cat_trans[cat][k]
/tmp/ipykernel_13151/2033497012.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_2nd[cat][train_2nd[cat]==k] = cat_trans[cat][k]
/tmp/ipykernel_13151/2033497012.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-

...

bank_id 61
income_type 6
rehabilitation 4
rate_rank 80
bank_cluster 5
cluster_min_rate 6
purpose 8
employment_type 4
houseown_type 4
******************** 2nd Classication Data Prepared! ********************
**************************************************
2nd Classication Start!
**************************************************
********************  XGBoost 2nd Classification is Stared ,,,,,,,,,,  ********************


/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda:4
  warnings.warn(f"Device used : {self.device}")


********************  XGBoost 2nd Classification is Done!  ********************
********************  TabNet 2nd Classification is Stared ,,,,,,,,,,  ********************


/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.44327 |  0:01:29s
********************  TabNet 2nd Classification is Done!  ********************


In [5]:


print('*'*50)
print('2nd Classication Soft Voting Ensemble is Proceeding ,,,,,,,,')
print('*'*50)

pred_prob_valid_xgb = clf_xgb.predict_proba(X_valid)

pred_prob_valid_tab = clf_tab.predict_proba(X_valid)

pred_prob_valid_xgb_else = clf_xgb.predict_proba(X_valid_else)

pred_prob_valid_tab_else = clf_tab.predict_proba(X_valid_else)
    
def proba_weighted_mean_best_pred(thres,x,y) :
    wm = (thres*x[:,1] + (100-thres)*y[:,1])/100
                                
    f1_best_thres_valid_ = 0.46

    return thres_2(f1_best_thres_valid_,wm)                  
                                
                                
                                



test_2nd['pred'] = proba_weighted_mean_best_pred(0.05*100, pred_prob_valid_tab, pred_prob_valid_xgb)



####### 1st & 2nd  ########





print('*'*50)
print('2nd Classication Soft Voting Ensemble is Done!')
print('*'*50)

print('*'*50)
print('Making Submission File ,,,,,,,,,')
print('*'*50)
    
test_2nd_else['pred'] = 0
final_2nd = pd.concat([test_2nd,test_2nd_else], ignore_index=True)

final_test = final_2nd.loc[:,['application_id','product_id','pred']]
submission_ = pd.read_csv('/Data2/folder_ti/data/submission/submission.csv')
submission = submission_.drop(['is_applied'],axis=1,inplace=False)
submission = pd.merge(submission,final_test,on=['application_id','product_id'],how='left')
submission.rename(columns={'pred':'is_applied'},inplace=True)
submission.to_csv('/Data2/folder_ti/data/submission/sub_test.csv')
    

print('*'*50)
print('Making Submission File is Done!!!!')
print('*'*50)




**************************************************
2nd Classication Soft Voting Ensemble is Proceeding ,,,,,,,,
**************************************************
**************************************************
2nd Classication Soft Voting Ensemble is Done!
**************************************************
**************************************************
Making Submission File ,,,,,,,,,
**************************************************
**************************************************
Making Submission File is Done!!!!
**************************************************


# 실제 Submission 파일 제작

In [1]:
!python ensemble_submission.py

Available devices  5
Device: cuda:4
Current cuda device: 4
**************************************************
1st classication preprocessing,,,,,,
**************************************************
100%|█████████████████████████████████████████████| 4/4 [00:00<00:00, 62.88it/s]
purpose 8
gender 3
income_type 6
employment_type 4
houseown_type 4
rehabilitation 4
****************** preprocessing end! ******************
********************  XGBoost 1st classification is stared!  ********************
********************  XGBoost 1st classification is done!  ********************
********************  TabNet 1st classification is stared!  ********************
epoch 0  | loss: 0.67267 |  0:00:12s
epoch 1  | loss: 0.63534 |  0:00:24s
epoch 2  | loss: 0.63109 |  0:00:35s
epoch 3  | loss: 0.62977 |  0:00:47s
epoch 4  | loss: 0.62747 |  0:00:59s
epoch 5  | loss: 0.62536 |  0:01:10s
epoch 6  | loss: 0.62379 |  0:01:22s
epoch 7  | loss: 0.62627 |  0:01:34s
epoch 8  | loss: 0.62411 |  0:01:45s
epoc

epoch 9  | loss: 0.34427 |  0:14:37s
epoch 10 | loss: 0.34224 |  0:16:07s
epoch 11 | loss: 0.34082 |  0:17:35s
epoch 12 | loss: 0.34035 |  0:19:04s
epoch 13 | loss: 0.33971 |  0:20:34s
epoch 14 | loss: 0.33958 |  0:22:03s
epoch 15 | loss: 0.33856 |  0:23:32s
epoch 16 | loss: 0.33862 |  0:25:00s
epoch 17 | loss: 0.33824 |  0:26:27s
epoch 18 | loss: 0.33773 |  0:27:54s
epoch 19 | loss: 0.3375  |  0:29:24s
epoch 20 | loss: 0.33755 |  0:30:56s
epoch 21 | loss: 0.33722 |  0:32:32s
epoch 22 | loss: 0.33699 |  0:34:01s
epoch 23 | loss: 0.33861 |  0:35:29s
epoch 24 | loss: 0.33747 |  0:36:57s
epoch 25 | loss: 0.33719 |  0:38:25s
epoch 26 | loss: 0.33637 |  0:39:53s
epoch 27 | loss: 0.33684 |  0:41:21s
epoch 28 | loss: 0.33651 |  0:42:46s
epoch 29 | loss: 0.33611 |  0:44:16s
epoch 30 | loss: 0.33654 |  0:45:42s
epoch 31 | loss: 0.33595 |  0:47:06s
epoch 32 | loss: 0.33622 |  0:48:31s
epoch 33 | loss: 0.33652 |  0:49:55s
epoch 34 | loss: 0.33578 |  0:51:14s
epoch 35 | loss: 0.33546 |  0:52:39s
e